# Phase Estimation

## Overview

- Type: Estimation task
- Time complexity: $O(1/\epsilon)$ for most algorithms, with estimation error $\epsilon$. 
- Approachability: Medium
- Related topics: Phase kickback, Shor's algorithm, Optical interferometry, Quantum Fourier Transform
- Applications: Factoring, eigenvalue estimation, solving linear systems, amplitude estimation, quantum mechanics

In the context of quantum computing, *phase estimation* refers to the problem of estimating of the eigenvalue of a unitary operator given an eigenvector. Algorithms which solve this problem are known as *phase estimation algorithms*. Often in practice, *phase estimation* is used to refer either to the problem or the algorithm interchangeably. 

## Problem statement

For a unitary operation $U$, and associated eigenvector $\vert u \rangle$, there is a $\theta \in [0,1)$ such that

$$
U \vert u \rangle = e^{2\pi i \theta} \vert u \rangle.
$$

Given oracular access to a unitary $U$, controlled versions thereof, and an eigenvector $\vert u \rangle$ prepared on a quantum register to fidelity $F$, determine $\theta$ to accuracy $\epsilon$. We refer to $\theta$ as the *phase*. 

## Algorithms

There are many Quantum Phase Estimation Algorithms (QPEA). In this article we will focus on the following incomplete list.  

- Standard QPEA
- QPEA via Hadamard Test
- Kitaev QPEA
- Fast QPEA (Svore, Hastings, Freedman)
- Bayesian QPEA 

### High level description

Phase estimation algorithms (PEAs) employ phase kickback and quantum interference to measure the desired phase $\theta$ on an auxiliary quantum register. Various degrees of classical post processing are required following the interference measurements. PEAs can be broadly categorized as non-iterative or iterative. Standard phase estimation is the best known of the non-iterative algorithms and requires no classical processing. Iterative PEAs typically use only a single auxiliary qubit and require multiple rounds, sometimes adaptive, and might require significant classical computation.

All but the simplest PEAs achieve a $O(1/\epsilon)$ scaling in accuracy, which is provably optimal in the general setting.

## Standard Phase Estimation Algorithm (Nielsen and Chuang)

### Overview

As the name suggests, the most commonly referred to algorithm for phase estimation, often simply abbreviated to "phase estimation." In contrast to "iterative" methods, this approach requires (with high probability) only one execution of the circuit below, but a number of auxiliary qubits proportional to the desired bits of precision. An $n$ bit approximation is of the phase is produced on an auxiliary register.

### Characteristics

Type: Single shot \
Qubit overhead: $O(\log(1/\epsilon) + \log(1/\delta))$. \
Runtime: $O(1/\epsilon)$ calls to the controlled-$U$. \
Works with superposition input?: Yes


### Circuit diagram

### Pseudocode (Nielsen and Chuang pg. 235)



### Qiskit Implementation

In [ ]:
import numpy as np

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library import QFT


def create_qpe_circuit(theta, num_qubits):
    """Creates a QPE circuit given theta and num_qubits."""

    # Step 1: Create a circuit with two quantum registers and one classical register.
    first = QuantumRegister(
        size=num_qubits, name="first"
    )  # the first register for phase estimation
    second = QuantumRegister(
        size=1, name="second"
    )  # the second register for storing eigenvector |psi>
    classical = ClassicalRegister(
        size=num_qubits, name="readout"
    )  # classical register for readout
    qpe_circuit = QuantumCircuit(first, second, classical)

    # Step 2: Initialize the qubits.
    # All qubits are initialized in |0> by default, no extra code is needed to initialize the first register.
    qpe_circuit.x(
        second
    )  # Initialize the second register with state |psi>, which is |1> in this example.

    # Step 3: Create superposition in the first register.
    qpe_circuit.barrier()  # Add barriers to separate each step of the algorithm for better visualization.
    qpe_circuit.h(first)

    # Step 4: Apply a controlled-U^(2^j) black box.
    qpe_circuit.barrier()
    for j in range(num_qubits):
        qpe_circuit.cp(
            theta * 2 * np.pi * (2**j), j, num_qubits
        )  # Theta doesn't contain the 2 pi factor.

    # Step 5: Apply an inverse QFT to the first register.
    qpe_circuit.barrier()
    qpe_circuit.compose(QFT(num_qubits, inverse=True), inplace=True)

    # Step 6: Measure the first register.
    qpe_circuit.barrier()
    qpe_circuit.measure(first, classical)

    return qpe_circuit

### Phase Estimation with the Hadamard test

#### Overview 

The Hadamard test serves as a phase estimation algorithm. See the Hadamard Test article for further information.

#### Characteristics

Type: Iterative \
Qubit overhead: $1$ ancilla \
Runtime: $O(1/\epsilon^2)$ iterations. \
Eigenvector superpositions?: No

#### Remarks

Possibly the simplest QPEA, but with quadratically worse scaling than other approaches in the accuracy. 

### Qiskit Implementation

The following Hadamard test function is taken from the Hadamard test page.

In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library.standard_gates import SwapGate

def hadamard_circuit(U, add_s_gate=False, measure=True):
    """
    Args:
        U (Gate): Unitary gate to measure expectation value of
        add_s_gate (bool): Whether to measure real (False) or imaginary
            (True) parts of the expectation value
        measure (bool): Whether to measure the auxiliary qubit
    Returns:
        QuantumCircuit that implements the Hadamard test
    """
    # Initialize registers and circuit
    aux = QuantumRegister(1, 'aux')
    main = QuantumRegister(U.num_qubits, 'q')
    circuit = QuantumCircuit(aux, main)
    
    # Construct controlled-U gate
    cU = U.control(1)
    
    # Add gates to circuit
    circuit.h(aux)
    circuit.append(cU, aux[:] + main[:])
    if add_s_gate:
        circuit.s(aux)
    circuit.h(aux)
    
    # Add measurement if option specified
    if measure:
        creg = ClassicalRegister(1, 'c')
        circuit.add_register(creg)
        circuit.measure(aux, creg)
    
    return circuit

# Draw an example
circuit = hadamard_circuit(SwapGate(), add_s_gate=True, measure=True)
circuit.draw()

In [ ]:
from qiskit.primitives import Sampler

def phase_estimation_primitive(U, precision = 10**-2):
    """
    Args:
        U (Gate): Unitary gate to measure expectation value of
        add_s_gate (bool): Whether to measure real (False) or imaginary
            (True) parts of the expectation value
        measure (bool): Whether to measure the auxiliary qubit
    Returns:
        QuantumCircuit that implements the Hadamard test
    """
Nshots = 1/precision**2
U = SwapGate()

for part in ("Real", "Imaginary"):
    # Build circuit
    circuit = hadamard_circuit(
        U,
        add_s_gate=(False if part=="Real" else True),
        measure=True
    )

    # Run circuit and get qasi-probability distribution
    results = Sampler().run(
        circuit,
        shots=N
    ).result().quasi_dists[0]

    # Get qasi-probability of measuring 0 (default to 0 if not in distribution)
    p0 = results.get(0, 0)

    # Calculate estimate
    estimate = ((2 * p0) - 1) * (1 if part=="Real" else -1)
    print(f"{part} part of expectation value: {estimate:.3f}")

### Kitaev Phase Estimation Algorithm

#### Overview

The Kitaev phase estimation algorithm (KPEA) is simply the Hadamard test repeated with certain multiples of the phase, followed by classical post processing on the resulting data. More specifically, instead of estimating 

#### Characteristics

Type: Iterative \
Qubit overhead: $1$ ancilla \
Runtime: ??? \
Eigenvector superpositions?: No

#### Circuit diagram

#### Pseudocode

### Qiskit Implementation

In [ ]:
import numpy as np


## Applications
- Order finding, thereby integer factorization (Shor's algorithm)
- Solving linear systems of equations (HHL algorithm)
- Quantum simulation

### See also

- 

In [ ]:
import qiskit.tools.jupyter

%qiskit_version_table